In [29]:
import numpy as np
import pandas as pd

from scipy.optimize import linprog

'''
Решение транспортной задачи где X это ограничение по ресурсам, Y требования по заполнению.
P -  платёжная матрица

Расчитать можно два варианта оптимизации:
1 Если Y количество работ то можно узнать минимальную цену их выполнения.
2 Если Y бюджет на работы то можно узнать максимальное число выполненных работ.
'''


# расчёт
def calcLinprog(optMax = 0, **g):

    # optMax = 1 #  максимизирует итоговую сумму, беря максимум ресурсов
    # optMax = 0 #  минимизирует сумму значений по строке, минимизирует ресурсы

    #P = np.array([7,8,1,2,6,5,4,3])
    #X = [100,100,100,100] # отсюда берём
    #Y = [160,150] # это равно
    P=g['P']
    X=g['X']
    Y=g['Y']

    lenP = len(P)
    lenX = len(X)
    lenY = len(Y)

    ub=np.tile(np.eye(lenX, dtype=int),lenY) # это x1 x2 x3 x4
    eq=np.repeat(np.eye(lenY, dtype=int), lenX, axis=1)  # это у1

    if optMax == 1:
        c=np.array(np.repeat(np.eye(1, dtype=int), lenP, axis=1))*(-1)
        eq = eq*P
    else:
        c=P
      
    res = dict(linprog(c, ub, X, eq, Y))
    res.update({'price':(res['x']*g['P']).reshape(lenY,lenX)})
    res.update({'res':res['x'].reshape(lenY,lenX)})
    return (res)




def getPDfromXLSX(wfile):
    xl = pd.ExcelFile(wFile)
    
    dP = pd.read_excel(xl, "price", index_col=[0])
    dX  = pd.read_excel(xl, "available", index_col=[0])
    dY  = pd.read_excel(xl, "org_plan", index_col=[0])
    d={}
    d['P']=dP.to_numpy().ravel()
    d['X']=dX.to_numpy().ravel()
    d['Y']=dY.to_numpy().ravel()
    
    d['pColumns']=dP.columns.values
    d['pIndex']=dP.index.values
    
    d['lenX']=len(d['pColumns'])
    d['lenY']=len(d['pIndex'])
    return d


# загрузка из файла
def formFromPD(price,data):

    inPD = pd.DataFrame(price, columns=data['pColumns'], index = data['pIndex'])
    
    outPD=inPD.copy()
    
    # to row
    
    outPD['SUMM']= inPD.sum(axis=1)
    
    outPD['MIN']= np.amin(inPD[inPD>0],axis=1)
    outPD['MAX']= inPD.max(axis=1)
    
    # to index
    
    outPD.loc['SUMM'] = inPD.sum(axis=0)
    
    outPD.loc['MIN']= np.amin(inPD[inPD>0],axis=0)
    outPD.loc['MAX']= inPD.max(axis=0)
    outPD.at['SUMM', 'SUMM'] =  inPD.to_numpy().sum()

    return(outPD)


# вывод в файл
def setPDtoXLSX(dPDs,oFile):
    with pd.ExcelWriter(oFile) as writer:  # doctest: +SKIP
        for dPD, sheet_name in dPDs:
            dPD.to_excel(writer, sheet_name=sheet_name)







In [30]:
import os
#projectName = 'pr_42'
inFileName = 'PR_42_20200504.xlsx'
ouFileName = 'PR_42_OUT_20200504_02.xlsx'
inFile=os.path.join(homeDir,inFileName)
ouFile=os.path.join(homeDir,ouFileName)


# загружаем данные
data = getPDfromXLSX(inFile)
# обрабатываем данные
optim = calcLinprog(**data, optMax=0)

print(optim)

e1=formFromPD(optim['price'],data)

e2=formFromPD(optim['res'],data)



t=([e1,'price'],[e2,'work'])

setPDtoXLSX(t,ouFile)

{'x': array([ 0.,  0.,  0.,  0.,  0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 50., 50.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,
        0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,
        0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0., 50.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,  0.,  0.,  0.]), 'fun': 1800.0, 'slack': array([150.,   0., 200., 200.,   0.,   0., 150., 100.]), 'con': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'status': 0, 'message': 'Optimization terminated successfully.', 'nit': 120, 'success': True, 'price': array([[  0.,   0.